In [75]:
from db_connection import db_query, db_query_dataframe, read_query, update_query

DB_SETTINGS = 'host=localhost dbname=addicted \
    user=addicted_r password=addicted_r port=3456'

query = read_query('sqls/story_processing.sql')
query = update_query(query, params={'start_date': '2019-04-01', 'end_date': '2020-09-14'})
data = db_query_dataframe(DB_SETTINGS, query)

Database connected


In [76]:
data.head()

,Date,Total Stories,Stories to Review,Approved Stories,Rejected Stories,Not Reviewed Stories,AI Rejected Stories,Total Episodes,Episodes to Review,Approved Episodes,Rejected Episodes,Not Reviewed Episodes,AI Rejected Episodes
0,2019-04-01,171,96,26,70,0,75,520,399.0,101.0,298.0,NaN,121.0
1,2019-04-02,35,17,3,14,0,18,77,52.0,11.0,41.0,NaN,25.0
2,2019-04-05,1,0,0,0,0,1,1,0.0,NaN,NaN,NaN,1.0
3,2019-04-08,1,1,1,0,0,0,4,NaN,4.0,NaN,NaN,NaN
4,2019-04-10,1,1,1,0,0,0,1,NaN,1.0,NaN,NaN,NaN


In [77]:
import datetime

data['period'] = data['Date'].apply(lambda x: x.strftime("%B") + ' ' + str(x.year))
data['sort'] = data['Date'].apply(lambda x: x.month + x.year*100)

In [78]:
data.drop(['Date'], axis='columns', inplace=True)

In [79]:
grouped_data = data.groupby(['period', 'sort']).sum().sort_values('sort').reset_index()

In [80]:
grouped_data

,period,sort,Total Stories,Stories to Review,Approved Stories,Rejected Stories,Not Reviewed Stories,AI Rejected Stories,Total Episodes,Episodes to Review,Approved Episodes,Rejected Episodes,Not Reviewed Episodes,AI Rejected Episodes
0,April 2019,201904,299,173,46,127,0,126,887,641.0,190.0,489.0,0.0,208.0
1,May 2019,201905,410,191,25,166,0,219,1011,655.0,101.0,571.0,0.0,339.0
2,June 2019,201906,443,178,52,126,0,265,1171,677.0,206.0,502.0,0.0,463.0
3,July 2019,201907,341,149,29,120,0,192,949,598.0,109.0,501.0,0.0,339.0
4,August 2019,201908,481,214,45,169,0,267,1313,836.0,191.0,645.0,0.0,477.0
5,September 2019,201909,311,119,28,91,0,192,784,467.0,102.0,383.0,0.0,299.0
6,October 2019,201910,435,163,35,128,0,272,1083,676.0,152.0,524.0,0.0,407.0
7,November 2019,201911,419,188,39,149,0,231,998,663.0,183.0,480.0,0.0,335.0
8,December 2019,201912,361,219,45,173,1,142,944,723.0,174.0,577.0,4.0,189.0
9,January 2020,202001,500,278,37,240,1,222,1286,995.0,139.0,847.0,9.0,291.0


In [81]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Not Reviewed', x=grouped_data['period'], y=grouped_data['Not Reviewed Stories'], 
           text=grouped_data['Not Reviewed Stories'], textposition='auto'),
    go.Bar(name='AI Rejected', x=grouped_data['period'], y=grouped_data['AI Rejected Stories'],
          text=grouped_data['AI Rejected Stories'], textposition='auto'),
    go.Bar(name='Rejected', x=grouped_data['period'], y=grouped_data['Rejected Stories'],
          text=grouped_data['Rejected Stories'], textposition='auto'),
    go.Bar(name='Approved', x=grouped_data['period'], y=grouped_data['Approved Stories'],
          text=grouped_data['Approved Stories'], textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='stack', uniformtext_minsize=9, uniformtext_mode='hide')
fig.show()

In [82]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Not Reviewed', x=grouped_data['period'], y=grouped_data['Not Reviewed Episodes'], 
           text=grouped_data['Not Reviewed Episodes'], textposition='auto'),
    go.Bar(name='AI Rejected', x=grouped_data['period'], y=grouped_data['AI Rejected Episodes'],
          text=grouped_data['AI Rejected Episodes'], textposition='auto'),
    go.Bar(name='Rejected', x=grouped_data['period'], y=grouped_data['Rejected Episodes'],
          text=grouped_data['Rejected Episodes'], textposition='auto'),
    go.Bar(name='Approved', x=grouped_data['period'], y=grouped_data['Approved Episodes'],
          text=grouped_data['Approved Episodes'], textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='stack', uniformtext_minsize=9, uniformtext_mode='hide')
fig.show()

In [83]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Scatter(name='Stories to Review', x=grouped_data['period'], y=grouped_data['Stories to Review']),
    go.Scatter(name='Episodes to Review', x=grouped_data['period'], y=grouped_data['Episodes to Review'])
])
# Change the bar mode
fig.update_layout(barmode='stack', uniformtext_minsize=9, uniformtext_mode='hide')
fig.show()